# listm'

In [ ]:
//// test

open testing

## listm'

### append

In [ ]:
instance append list t =
    listm.append

In [ ]:
//// test

[ "a"; "b" ] ++ [ "c"; "d" ]
|> _assert_eq [ "a"; "b"; "c"; "d" ]

__assert_eq / actual: UH0_1 ("a", UH0_1 ("b", UH0_1 ("c", UH0_1 ("d", UH0_0)))) / expected: UH0_1 ("a", UH0_1 ("b", UH0_1 ("c", UH0_1 ("d", UH0_0))))


### collect

In [ ]:
inl collect forall t r. (fn : t -> list r) (items : list t) : list r =
    items
    |> listm.map fn
    |> listm.fold (++) []

### replicate

In [ ]:
inl replicate count x =
    listm.init count fun _ => x

### map4

In [ ]:
inl map4 f l1 l2 l3 l4 =
    inl rec loop l1 l2 l3 l4 acc =
        match l1, l2, l3, l4 with
        | (x1 :: xs1), (x2 :: xs2), (x3 :: xs3), (x4 :: xs4) =>
            loop xs1 xs2 xs3 xs4 (f x1 x2 x3 x4 :: acc)
        | _ => acc |> listm.rev
    loop l1 l2 l3 l4 []

### init_series

In [ ]:
inl init_series start end inc =
    inl total : f64 = conv ((end - start) / inc) + 1
    listm.init total (conv >> (*) inc >> (+) start)

In [ ]:
//// test

init_series 0 1 0.5
|> _assert_eq [ 0f64; 0.5; 1 ]

__assert_eq / actual: UH0_1 (0.0, UH0_1 (0.5, UH0_1 (1.0, UH0_0))) / expected: UH0_1 (0.0, UH0_1 (0.5, UH0_1 (1.0, UH0_0)))


### try_item

In [ ]:
inl rec try_item i = function
    | Cons (x, _) when i = 0 => Some x
    | Cons (_, xs) => try_item (i - 1) xs
    | Nil => None

In [ ]:
//// test

listm.init 10i32 id
|> try_item 9i32
|> _assert_eq (Some 9)

__assert_eq / actual: US0_0 9 / expected: US0_0 9


In [ ]:
//// test

listm.init 10i32 id
|> try_item 10i32
|> _assert_eq None

__assert_eq / actual: US0_1 / expected: US0_1


### item

In [ ]:
inl item i =
    try_item i >> optionm.value

In [ ]:
//// test

listm.init 10i32 id
|> item 9i32
|> _assert_eq 9

__assert_eq / actual: 9 / expected: 9


In [ ]:
//// test

fun () =>
    listm.init 10i32 id
    |> item 10i32
    |> ignore
|> _throws
|> optionm.map sm'.format_exception
|> _assert_eq (Some "System.Exception: Option does not have a value.")

__assert_eq / actual: US1_0 "System.Exception: Option does not have a value." / expected: US1_0 "System.Exception: Option does not have a value."


### try_item_

In [ ]:
let rec try_item_ i = function
    | Cons (x, _) when i = 0 => Some x
    | Cons (_, xs) => try_item_ (i - 1) xs
    | Nil => None

### item_

In [ ]:
inl item_ i =
    try_item_ i >> optionm.value

### sum

In [ ]:
inl sum list =
    list |> listm.fold (+) 0

In [ ]:
//// test

listm.init 10i32 id
|> sum
|> _assert_eq 45

__assert_eq / actual: 45 / expected: 45


### unzip

In [ ]:
inl unzip list =
    (([], []), list)
    ||> listm.fold fun (acc_x, acc_y) (x, y) =>
        x :: acc_x, y :: acc_y
    |> fun x, y =>
        x |> listm.rev, y |> listm.rev

In [ ]:
//// test

listm.init 10i32 id
|> listm.map (fun x => x, x)
|> unzip
|> fun x, y =>
    x |> sum
    |> _assert_eq 45

    y |> sum
    |> _assert_eq 45

__assert_eq / actual: 45 / expected: 45
__assert_eq / actual: 45 / expected: 45


### try_index_of

In [ ]:
inl try_index_of item list =
    inl rec loop i = function
        | [] => None
        | x :: xs =>
            if x = item
            then Some i
            else loop (i + 1) xs
    loop 0 list

inl index_of item =
    try_index_of item >> optionm.value

inl try_index_of_ item list =
    let rec loop i = function
        | [] => None
        | x :: xs =>
            if x = item
            then Some i
            else loop (i + 1) xs
    loop 0 list

inl index_of_ item =
    try_index_of_ item >> optionm.value

inl try_index_of__ item list =
    inl i = mut 0
    inl list = mut list
    inl result = mut None
    let rec loop () =
        match *list with
        | [] => result <- None
        | x :: xs =>
            if x = item
            then result <- Some *i
            else
                i <- *i + 1
                list <- xs
                loop ()
    loop ()
    *result

inl index_of__ item =
    try_index_of__ item >> optionm.value

In [ ]:
//// test

listm.init 10i32 id
|> index_of 5i32
|> _assert_eq 5i32

__assert_eq / actual: 5 / expected: 5


In [ ]:
//// test

listm.init 10i32 id
|> try_index_of 10i32
|> _assert_eq (None : option i32)

__assert_eq / actual: US0_1 / expected: US0_1


### try_find

In [ ]:
inl try_find fn list =
    inl rec loop = function
        | [] => None
        | x :: xs =>
            if fn x
            then Some x
            else loop xs
    loop list

inl try_find_ fn list =
    let rec loop = function
        | [] => None
        | x :: xs =>
            if fn x
            then Some x
            else loop xs
    loop list

In [ ]:
//// test

listm.init 10i32 id
|> try_find ((=) 5i32)
|> _assert_eq (Some 5i32)

__assert_eq / actual: US0_0 5 / expected: US0_0 5


In [ ]:
inl find x =
    try_find x >> optionm.value

inl find_ x =
    try_find_ x >> optionm.value

In [ ]:
//// test

listm.init 10i32 id
|> find ((=) 5i32)
|> _assert_eq 5i32

__assert_eq / actual: 5 / expected: 5


### choose

In [ ]:
inl choose f l =
    (l, [])
    ||> listm.foldBack fun x acc =>
        match f x with
        | Some y => y :: acc
        | None => acc

In [ ]:
//// test

listm.init 10i32 id
|> choose (fun x => if x % 2 = 0 then Some x else None)
|> _assert_eq [ 0; 2; 4; 6; 8 ]

__assert_eq / actual: UH0_1 (0, UH0_1 (2, UH0_1 (4, UH0_1 (6, UH0_1 (8, UH0_0))))) / expected: UH0_1 (0, UH0_1 (2, UH0_1 (4, UH0_1 (6, UH0_1 (8, UH0_0)))))


### filter

In [ ]:
inl filter forall t. (fn : t -> bool) (list : list t) : list t =
    (list, Nil)
    ||> listm.foldBack fun x acc =>
        if fn x then x :: acc else acc

### zip_with

In [ ]:
inl zip_with fn xs ys =
    inl rec loop acc xs ys =
        match xs, ys with
        | Cons (x, xs), Cons (y, ys) =>
            loop (fn x y :: acc) xs ys
        | _ => listm.rev acc
    loop [] xs ys

inl zip_with_ fn xs ys =
    let rec loop acc xs ys =
        match xs, ys with
        | Cons (x, xs), Cons (y, ys) =>
            loop (fn x y :: acc) xs ys
        | _ => listm.rev acc
    loop [] xs ys

In [ ]:
//// test

([ 1i32; 2; 3 ], [ 4; 5; 6 ])
||> zip_with (+)
|> _assert_eq [ 5; 7; 9 ]

__assert_eq / actual: UH0_1 (5, UH0_1 (7, UH0_1 (9, UH0_0))) / expected: UH0_1 (5, UH0_1 (7, UH0_1 (9, UH0_0)))


### zip

In [ ]:
inl zip xs ys =
    zip_with pair xs ys

inl zip_ xs ys =
    zip_with_ pair xs ys

In [ ]:
//// test

([ 1i32; 2; 3 ], [ 4i32; 5; 6 ])
||> zip
|> _assert_eq [ 1, 4; 2, 5; 3, 6 ]

__assert_eq / actual: UH0_1 (1, 4, UH0_1 (2, 5, UH0_1 (3, 6, UH0_0))) / expected: UH0_1 (1, 4, UH0_1 (2, 5, UH0_1 (3, 6, UH0_0)))


### indexed

In [ ]:
inl indexed list =
    (([], 0), list)
    ||> listm.fold fun (acc, i) x =>
        (i, x) :: acc, i + 1
    |> fst
    |> listm.rev

In [ ]:
//// test

listm.init 5i32 ((*) 2)
|> indexed
|> _assert_eq [ 0i32, 0; 1, 2; 2, 4; 3, 6; 4, 8 ]

__assert_eq / actual: UH0_1 (0, 0, UH0_1 (1, 2, UH0_1 (2, 4, UH0_1 (3, 6, UH0_1 (4, 8, UH0_0))))) / expected: UH0_1 (0, 0, UH0_1 (1, 2, UH0_1 (2, 4, UH0_1 (3, 6, UH0_1 (4, 8, UH0_0)))))


### group_by

In [ ]:
inl group_by fn list =
    (list, [])
    ||> listm.foldBack fun x acc =>
        inl xk = fn x
        inl found, new_acc =
            ((false, []), acc)
            ||> listm.fold fun (found, acc') (k, xs) =>
                if k = xk
                then true, (k, x :: xs) :: acc'
                else found, (k, xs) :: acc'
        if found
        then new_acc
        else (xk, [ x ]) :: new_acc

In [ ]:
//// test

listm.init 10i32 id
|> group_by (fun x => x % 2 = 0)
|> _assert_eq [ true, [ 0; 2; 4; 6; 8 ]; false, [ 1; 3; 5; 7; 9 ] ]

__assert_eq / actual: UH1_1
  (true, UH0_1 (0, UH0_1 (2, UH0_1 (4, UH0_1 (6, UH0_1 (8, UH0_0))))),
   UH1_1
     (false, UH0_1 (1, UH0_1 (3, UH0_1 (5, UH0_1 (7, UH0_1 (9, UH0_0))))), UH1_0)) / expected: UH1_1
  (true, UH0_1 (0, UH0_1 (2, UH0_1 (4, UH0_1 (6, UH0_1 (8, UH0_0))))),
   UH1_1
     (false, UH0_1 (1, UH0_1 (3, UH0_1 (5, UH0_1 (7, UH0_1 (9, UH0_0))))), UH1_0))


### forall'

In [ ]:
inl forall' fn (head :: tail) =
    (true, tail)
    ||> listm.fold fun acc x =>
        acc && x = head

In [ ]:
//// test

[ 1i32; 1; 1; 1; 1 ]
|> forall' ((=) 1i32)
|> _assert_eq true

__assert_eq / actual: true / expected: true


### last

In [ ]:
inl last list =
    list
    |> listm.rev
    |> item 0i32

In [ ]:
//// test

listm.init 10i32 id
|> last
|> _assert_eq 9

__assert_eq / actual: 9 / expected: 9


### try_pick

In [ ]:
inl try_pick fn list =
    inl rec body fn = function
        | [] => None
        | x :: xs =>
            match fn x with
            | Some y => Some y
            | None => loop xs
    and inl loop list =
        if var_is list |> not
        then body fn list
        else
            inl fn = join fn
            inl list = dyn list
            join body fn list
    loop list

In [ ]:
//// test

listm.init 10i32 id
|> try_pick (fun x => if x = 5i32 then Some x else None)
|> _assert_eq (Some 5i32)

__assert_eq / actual: US0_0 5 / expected: US0_0 5


### exists'

In [ ]:
inl exists' f x =
    inl length_x : i64 = x |> listm.length
    let rec loop i =
        if i >= length_x
        then false
        elif x |> item i |> f
        then true
        else loop (i + 1)
    loop 0

In [ ]:
//// test

[ 'a'; 'b'; 'c' ]
|> exists' fun x => x = 'b'
|> _assert_eq true

[ 'a'; 'b' ]
|> exists' fun x => x = 'c'
|> _assert_eq false

[]
|> exists' fun x => x = 'a'
|> _assert_eq false

__assert_eq / actual: true / expected: true
__assert_eq / actual: false / expected: false
__assert_eq / actual: false / expected: false


## fsharp

### list'

In [ ]:
nominal list' t = $"backend_switch `({ Fsharp : $'`t list'; Python : $'list' })"

### empty'

In [ ]:
inl empty' forall t. () : list' t =
    $'[]'

### cons'

In [ ]:
inl cons' forall t. (head : t) (tail : list' t) : list' t =
    backend_switch {
        Fsharp = fun () => $'!head :: !tail ' : list' t
        Python = fun () =>
            $'!tail.insert(0, !head)'
            $'!tail ' : list' t
    }

### rev'

In [ ]:
inl rev' forall t. (items : list' t) : list' t =
    backend_switch {
        Fsharp = fun () => items |> $'List.rev' : list' t
        Python = fun () => $'list(reversed(!items))' : list' t
    }

### box

In [ ]:
inl box forall t. (list : list t) : list' t =
    (list, empty' ())
    ||> listm.foldBack fun x acc =>
        acc |> cons' x

### fold'

In [ ]:
inl fold' forall t u. (fn : t -> u) (init : list u) (list : list' t) : list u =
    backend_switch {
        Fsharp = fun () =>
            (init, list)
            ||> $'List.fold' join fun acc x => Cons (fn x, acc)
            : list u
        Python = fun () =>
            inl init = init |> box
            $'r = !init '
            inl list = list |> rev'
            $'for x in !list: r = [!fn(x)] + r'
            inl init : list u = Nil
            inl cons (a : u) b = Cons (a, b)
            $'r_ = !init '
            $'for x in r: r_ = !cons (x)(r_)'
            $'r_' : list u
    }

### fold_back'

In [ ]:
inl fold_back' forall t u. (fn : t -> u) (list : list' t) (init : list u) : list u =
    backend_switch {
        Fsharp = fun () =>
            (list, init)
            ||> $'List.foldBack' join fun x acc => Cons (fn x, acc)
            : list u
        Python = fun () =>
            list
            |> rev'
            |> fold' fn init
    }

### filter'

In [ ]:
inl filter' forall t. (fn : t -> bool) (list : list' t) : list' t =
    backend_switch {
        Fsharp = fun () => list |> $'"List.filter !fn"' : list' t
        Python = fun () => $'list(filter(!fn, !list))' : list' t
    }

### map

In [ ]:
inl map forall t u. (fn : t -> u) (list : list' t) : list' u =
    backend_switch {
        Fsharp = fun () => list |> $'List.map' fn : list' u
        Python = fun () => $'list(map(!fn, !list))' : list' u
    }

### unbox

In [ ]:
inl unbox forall t. (list : list' t) : list t =
    (list, Nil)
    ||> fold_back' id

### distinct'

In [ ]:
// preserve order
inl distinct' forall t. (list : list' t) : list' t =
    backend_switch {
        Fsharp = fun () => list |> $'List.distinct' : list' t
        Python = fun () =>
            $'x = list(set(!list))'
            $'x.sort(key=!list.index)'
            $'x' : list' t
    }

In [ ]:
//// test
///! fsharp
///! cuda

[ "1"; "2"; "2"; "3" ]
|> box
|> distinct'
|> unbox
|> _assert_eq [ "1"; "2"; "3" ]

.py output (Cuda):
__assert_eq / actual: UH0_1(v0='1', v1=UH0_1(v0='2', v1=UH0_1(v0='3', v1=UH0_0()))) / expected: UH0_1(v0='1', v1=UH0_1(v0='2', v1=UH0_1(v0='3', v1=UH0_0())))



.fsx output:
__assert_eq / actual: UH0_1 ("1", UH0_1 ("2", UH0_1 ("3", UH0_0))) / expected: UH0_1 ("1", UH0_1 ("2", UH0_1 ("3", UH0_0)))


### to_array'

In [ ]:
inl to_array' forall t. (items : list' t) : array_base t =
    backend_switch {
        Fsharp = fun () => items |> $'List.toArray' : array_base t
        Python = fun () => $'(cp if cuda else np).array(!items)' : array_base t
    }